In [ ]:
"# Databricks notebook source\n# MAGIC %md\n# MAGIC # ETL Process for Superstore Sales Data\n# MAGIC This notebook performs an ETL process on Superstore sales data using PySpark. It loads data from Unity Catalog, transforms it, and writes the results back to Unity Catalog.\n\n# COMMAND ----------\n\nimport logging\nfrom pyspark.sql import SparkSession\nfrom pyspark.sql.functions import col, concat, to_date, when, datediff, year, sum, avg\nfrom pyspark.sql.types import DoubleType\n\n# Initialize logging\nlogging.basicConfig(level=logging.INFO)\nlogger = logging.getLogger(__name__)\n\n# Assume the Spark session is already initialized as 'spark'\n\n# COMMAND ----------\n\ndef load_data():\n    try:\n        # Load data from Unity Catalog tables\n        orders_central_df = spark.table(\"catalog.db.orders_central\")\n        quota_df = spark.table(\"catalog.db.quota\")\n        orders_west_df = spark.table(\"catalog.db.orders_west\")\n        returns_all_df = spark.table(\"catalog.db.returns_all\")\n        orders_east_df = spark.table(\"catalog.db.orders_east\")\n        orders_south_2015_df = spark.table(\"catalog.db.orders_south_2015\")\n        \n        logger.info(\"Data loaded successfully from Unity Catalog tables.\")\n        return orders_central_df, quota_df, orders_west_df, returns_all_df, orders_east_df, orders_south_2015_df\n    except Exception as e:\n        logger.error(f\"Error loading data: {e}\")\n        raise\n\n# COMMAND ----------\n\ndef transform_data(orders_central_df, quota_df, orders_west_df, returns_all_df, orders_east_df, orders_south_2015_df):\n    try:\n        # Data Standardization\n        orders_central_df = orders_central_df.withColumn(\"Order Date\", to_date(concat(col(\"Order Day\"), col(\"Order Month\"), col(\"Order Year\")), \"ddMMyyyy\"))\n        orders_central_df = orders_central_df.withColumn(\"Ship Date\", to_date(concat(col(\"Ship Day\"), col(\"Ship Month\"), col(\"Ship Year\")), \"ddMMyyyy\"))\n        orders_central_df = orders_central_df.withColumnRenamed(\"Discounts\", \"Discount\").withColumnRenamed(\"Product\", \"Product Name\")\n        \n        # Replace State Names with Abbreviations\n        state_abbreviations = {\"California\": \"CA\", \"New York\": \"NY\"}  # Example mapping\n        orders_central_df = orders_central_df.replace(state_abbreviations, subset=[\"State\"])\n        \n        # Data Cleaning\n        orders_central_df = orders_central_df.filter(orders_central_df[\"Order ID\"].isNotNull())\n        orders_central_df = orders_central_df.fillna({\"Discount\": 0})\n        orders_central_df = orders_central_df.withColumn(\"Sales\", col(\"Sales\").cast(DoubleType()))\n        \n        # Pivoting and Consolidation\n        quota_df = quota_df.selectExpr(\"Region\", \"stack(4, '2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`) as (Year, Quota)\")\n        \n        # Combine Order Datasets\n        combined_orders_df = orders_central_df.union(orders_west_df).union(orders_east_df).union(orders_south_2015_df)\n        \n        # Calculated Fields and Enhancements\n        combined_orders_df = combined_orders_df.withColumn(\"Returned?\", when(col(\"Return Reason\").isNotNull(), \"Yes\").otherwise(\"No\"))\n        combined_orders_df = combined_orders_df.withColumn(\"Days to Ship\", datediff(col(\"Ship Date\"), col(\"Order Date\")))\n        combined_orders_df = combined_orders_df.withColumn(\"Year of Sale\", year(col(\"Order Date\")))\n        \n        # Apply Business Rules\n        combined_orders_df = combined_orders_df.filter(~((col(\"Discount\") >= 17) & (col(\"Discount\") < 18)))\n        \n        logger.info(\"Data transformation completed successfully.\")\n        return combined_orders_df, quota_df\n    except Exception as e:\n        logger.error(f\"Error during data transformation: {e}\")\n        raise\n\n# COMMAND ----------\n\ndef write_data(combined_orders_df):\n    try:\n        # Write to Unity Catalog target tables\n        combined_orders_df.write.format(\"delta\").mode(\"overwrite\").saveAsTable(\"catalog.db.superstore_sales\")\n        \n        performance_df = combined_orders_df.groupBy(\"Region\", \"Year of Sale\").agg(\n            sum(\"Profit\").alias(\"Total Profit\"),\n            sum(\"Sales\").alias(\"Total Sales\"),\n            sum(\"Quantity\").alias(\"Total Quantity\"),\n            avg(\"Discount\").alias(\"Average Discount\")\n        )\n        performance_df.write.format(\"delta\").mode(\"overwrite\").saveAsTable(\"catalog.db.annual_regional_performance\")\n        \n        logger.info(\"Data written successfully to Unity Catalog tables.\")\n    except Exception as e:\n        logger.error(f\"Error writing data: {e}\")\n        raise\n\n# COMMAND ----------\n\ndef main():\n    try:\n        orders_central_df, quota_df, orders_west_df, returns_all_df, orders_east_df, orders_south_2015_df = load_data()\n        combined_orders_df, quota_df = transform_data(orders_central_df, quota_df, orders_west_df, returns_all_df, orders_east_df, orders_south_2015_df)\n        write_data(combined_orders_df)\n    except Exception as e:\n        logger.error(f\"ETL process failed: {e}\")\n\n# COMMAND ----------\n\nif __name__ == \"__main__\":\n    main()\n"
